In [ ]:
import gdsfactory as gf
from blocks import *
from comb_drive_tuning import *

In [ ]:
from gdsfactory.generic_tech import get_generic_pdk
from gdsfactory.typings import Layer
from gdsfactory.technology import (
    LayerMap,
)
generic_pdk = get_generic_pdk()
class MYLayerMap(LayerMap):
    WG: Layer = (1, 0)
    DEEP_ETCH: Layer = (3, 6)
    SHALLOW_ETCH: Layer = (2, 6)
    ALD_CORE: Layer = (5, 0)
    ALD_ETCH_EBL: Layer = (3, 8)
    ALD_ETCH_PL: Layer = (3, 10)
    DEEP_ETCH_EBL: Layer = (10,0)
    DEEP_ETCH_PL: Layer = (9,0)
    MTOP: Layer = getattr(generic_pdk.layers, "MTOP")
    PADDING: Layer = (67,0)
    SLAB150: Layer = (2, 0)
    FLOORPLAN: Layer = getattr(generic_pdk.layers, "FLOORPLAN")
    MARKER: Layer = (66, 0)
pdk1 = gf.Pdk(
    name="fab1",
    layers=MYLayerMap,
    cross_sections=generic_pdk.cross_sections,
    layer_views=generic_pdk.layer_views,
    cells=generic_pdk.cells,
)
pdk1.activate()


In [ ]:
def make_ALD_EBL_mask(c):
    reg = gf.kdb.Region()
    reg += c.get_region(layer=gf.get_layer('DEEP_ETCH'))
    reg += c.get_region(layer=gf.get_layer('WG'))
    reg -= c.get_region(layer=gf.get_layer('ALD_CORE'))
    c.add_polygon(reg, layer=gf.get_layer('ALD_ETCH_EBL'))
def make_ALD_PL_mask(c):
    c.add_polygon(c.get_region(layer=gf.get_layer('MTOP')), layer=gf.get_layer('ALD_ETCH_PL'))

In [ ]:
xs_metal_wire = metal_wire(core_width=7, wire_width=5.5, mask_offset=10.0)
xs_metal_wire_wide = metal_wire(core_width=60, wire_width=50, mask_offset=10.0)
xs = cross_section_with_sleeves(0.43,10,radius=5,radius_min=5)
xs_routing = cross_section_with_sleeves(0.43,10,radius=20, radius_min=20)
Xtrans = gf.path.transition(cross_section1=xs_metal_wire, cross_section2=xs_metal_wire_wide, width_type="linear",offset_type="linear",)
gc_spec = partial(gf.components.grating_coupler_elliptical,wavelength=1.55,nclad=1,neff=2.3,n_periods=60,fiber_angle=8)
my_coupler_spec = partial(my_coupler,coupler=gc_spec,waveguide_width=0.43)

In [ ]:
def beam_with_ring_resonator_3_combs(beam_length=50, beam_width=0.3,spring_length=50, spring_width=0.16,finger_num = 70, finger_num_sub=30, waveguide_spacing=50):
    finger_length = 3.2
    finger_gap = 0.3
    a_c = 3
    finger_thickness = 0.2  
    x_span_resonator = 10
    
    spring_spec = partial(spring_pair_anchor_outside,spring_length=spring_length,spring_width=spring_width,mask_offset=10,spring_separation=4, open=['left','right'])    
    c = gf.Component()
    
    bridge1_ref = c << bridge(mxn=(8,max(85-beam_length,5)),open=['left'])
    bridge2_ref = c << bridge(mxn=(4,3),open=['right','left','bottom','top'])
    bridge2_ref.connect("E1", bridge1_ref.ports["W1"],allow_width_mismatch=True,allow_layer_mismatch=True)

    finger_sub = combdrive_fingers(fingers=finger_num_sub,finger_length=finger_length,finger_gap=finger_gap,thickness=finger_thickness,base_thickness=0,a_c=a_c,mask_offset=10)
    comb1_ref = c << finger_sub
    mfs_1_ref = c << movable_finger_support(length=(spring_spec().info['frame_length']-4)/2,mask_offset=10,open=["bottom","left"])
    
    mfs_1_ref.connect("S1", bridge2_ref.ports["N1"],allow_width_mismatch=True)
    comb1_ref.connect("w1", mfs_1_ref.ports["E1"],allow_width_mismatch=True)
    comb1_ref.movey(2.22)
    fhs_sub1 = finger_hard_support_L(size=(10,finger_num_sub*(0.2+0.3)),mask_offset=7, metal_offset=1, metal_layer='MTOP')
    fhs_sub1_ref = c << fhs_sub1
    fhs_sub1_ref.connect("W1", comb1_ref.ports["e1"],allow_width_mismatch=True, allow_type_mismatch=True)
    
    comb2_ref = c << finger_sub
    mfs_2_ref = c << movable_finger_support(length=(spring_spec().info['frame_length']-4)/2,mask_offset=10,open=["top","left"])
    mfs_2_ref.connect("N1", bridge2_ref.ports["S1"],allow_width_mismatch=True)
    comb2_ref.connect("w1", mfs_2_ref.ports["E1"],allow_width_mismatch=True)
    comb2_ref.movey(-1.8)
    fhs_sub2 = finger_hard_support_L(size=(10,finger_num_sub*(0.2+0.3)),mask_offset=7, metal_offset=1, metal_layer='MTOP')
    fhs_sub2_ref = c << fhs_sub2
    fhs_sub2_ref.connect("E1", comb2_ref.ports["e1"],allow_width_mismatch=True, allow_type_mismatch=True)

    spring_ref = c << spring_spec()
    spring_ref.connect("e1", bridge2_ref.ports["W1"],allow_width_mismatch=True)

    # comb drive for stretching
    stretch_finger = combdrive_fingers(fingers=finger_num,finger_length=finger_length,finger_gap=finger_gap,thickness=finger_thickness,base_thickness=0,a_c=a_c,mask_offset=10)
    mfs_length = max(finger_num*(0.2+0.3),  spring_spec().info['frame_length'])

    if mfs_length > spring_spec().info['frame_length']:
        mfs_stretch_ref = c << movable_finger_support(length=spring_spec().info['frame_length'],mask_offset=10,open=["right",'bottom','top'])
        mfs_stretch_ref.connect("E1", spring_ref.ports["w1"],allow_width_mismatch=True)
        add_ = (mfs_length - spring_spec().info['frame_length'])/2
        mfs_stretch_ref_btm = c << bridge(mxn=(ceil(add_),3),open=['top'])
        mfs_stretch_ref_btm.connect("N1", mfs_stretch_ref.ports["S1"],allow_width_mismatch=True)
        mfs_stretch_ref_top = c << bridge(mxn=(ceil(add_),3),open=['bottom'])
        mfs_stretch_ref_top.connect("S1", mfs_stretch_ref.ports["N1"],allow_width_mismatch=True) 
    else:
        mfs_stretch_ref = c << movable_finger_support(length=spring_spec().info['frame_length'],mask_offset=10,open=["right"])
        mfs_stretch_ref.connect("E1", spring_ref.ports["w1"],allow_width_mismatch=True)
        
   
    stretch_comb_ref = c << stretch_finger
    stretch_comb_ref.connect("w1", mfs_stretch_ref.ports["W1"],allow_width_mismatch=True)
    fhs_stretch = finger_hard_support(size=(10,finger_num*(0.2+0.3)),mask_offset=10, metal_offset=1, metal_layer='MTOP')
    fhs_stretch_ref = c << fhs_stretch
    fhs_stretch_ref.connect("E1", stretch_comb_ref.ports["e1"],allow_width_mismatch=True, allow_type_mismatch=True)
    
    # pretty the deep etch mask by making them to a big rectangle
    create_deep_etch_mask(c, method='bbox', mask_offset=0)
    beam_spec = partial(doubly_clamped_beam_with_round_support,length=beam_length, width=beam_width,support_length=0.2,create_mask=True,mask_offset=3,layer='ALD_CORE')
    beam_ref = c << beam_spec()
    beam_ref.connect("w1", bridge1_ref.ports["E1"],allow_width_mismatch=True,allow_layer_mismatch=True,allow_type_mismatch=True)
    c.movey(origin=(beam_ref.ymax+beam_ref.ymin)/2,destination=0)
    ald_beam_attachment = gf.components.rectangle(size=(0.16, 1), layer='ALD_CORE')
    ald_beam_attachment_right = c << ald_beam_attachment
    ald_beam_attachment_right.connect("e1", beam_ref.ports["e1"],allow_width_mismatch=True,allow_layer_mismatch=True,allow_type_mismatch=True)
    ald_beam_attachment_left = c << ald_beam_attachment
    ald_beam_attachment_left.connect("e1", beam_ref.ports["w1"],allow_width_mismatch=True,allow_layer_mismatch=True,allow_type_mismatch=True)

    make_ALD_EBL_mask(c)
    

    spring_pad = pad(size=(400, 400), metal_offset=20,pad_layer='PADDING')
    spring_pad_ref = c << spring_pad
    spring_pad_ref.movex(origin=spring_pad_ref.xmax, destination=beam_ref.xmin-100)
    spring_pad_ref.movey(250)
    spring_pad_port_cp = spring_pad_ref.ports["S1"].copy()
    spring_pad_port_cp.center = (spring_pad_port_cp.center[0]+130, spring_pad_port_cp.center[1])
    p1 = gf.path.straight(length=80)
    p1 = p1.extrude(xs_metal_wire_wide)
    p1_ref = c << p1
    p1_ref.connect("e1", spring_pad_port_cp,allow_width_mismatch=True, allow_type_mismatch=True)
    wire_taper = gf.path.extrude_transition(gf.path.straight(25), Xtrans)
    wire_taper_ref = c << wire_taper
    wire_taper_ref.connect("e2", p1_ref.ports["e2"],allow_width_mismatch=True, allow_type_mismatch=True)
    
    # routing from spring beam to pad
    gf.routing.route_single(
        c,
        port1=spring_ref.ports['up_anchor'],
        port2=wire_taper_ref.ports['e1'],
        cross_section=xs_metal_wire,
        allow_width_mismatch=True,
        auto_taper=False,
        start_straight_length=100,
        )
    compressive_pad = U_shape_pad(p1_size=(200,1300), p2_size=(600,200), p3_size=(400,800), metal_offset=20)
    compressive_pad_ref = c << compressive_pad
    compressive_pad_ref.connect("e9", beam_ref.ports["w1"],allow_width_mismatch=True, allow_type_mismatch=True,allow_layer_mismatch=True)
    compressive_pad_ref.movex(-560).movey(300)
    
    pad_down_start_port = compressive_pad_ref.ports['e9'].copy()
    pad_down_start_port.center = (pad_down_start_port.center[0], fhs_sub2_ref.ports["N1"].center[1]-250)
    xs_fhs = metal_wire(core_width=10, wire_width=8, mask_offset=10.0)
    routing_with_mytaper(c, pad_down_start_port, fhs_sub2_ref.ports['N1'], xs_metal_wire_wide, xs_fhs, taper_length=60,tangent_offset=-30)
  
    pad_e10_moved = compressive_pad_ref.ports['e10'].copy()
    pad_e10_moved.center = (pad_e10_moved.center[0]-160, compressive_pad_ref.ports['e10'].center[1])
    gf.routing.route_single(
        c, 
        fhs_sub1_ref.ports['N1'], 
        pad_e10_moved, 
        cross_section=xs_metal_wire, 
        allow_width_mismatch=True, 
        auto_taper=False,
        start_straight_length=100,
        )
    
    stretch_pad = pad(size=(400, 300), metal_offset=20,pad_layer='PADDING')
    stretch_pad_ref = c << stretch_pad
    stretch_pad_ref.move(
        origin=stretch_pad_ref.center,
        destination=(spring_pad_ref.center[0],spring_pad_ref.center[1]-500)
    )
    stretch_pad_port = stretch_pad_ref.ports["E1"].copy()
    stretch_pad_port.center = (stretch_pad_port.center[0], fhs_stretch_ref.center[1])
    routing_with_mytaper(c, stretch_pad_port, fhs_stretch_ref.ports['W1'], xs_metal_wire_wide, fhs_stretch.cross_section, taper_length=10)
    

    
    
    ring_beam_spacer = vertical_spacer(length=0.1)
    ring_ref = c << ring_resonator_fill_middle(gap=0.3, radius=10, length_x=3,length_y=0,cross_section=xs,bend="bend_circular", x_span=x_span_resonator)
    spacer_ref = c << ring_beam_spacer
    spacer_ref.connect('p1', beam_ref.ports['s1'],allow_width_mismatch=True,allow_layer_mismatch=True,allow_type_mismatch=True)
    ring_ref.connect('p1', spacer_ref.ports['p2'],allow_width_mismatch=True,allow_layer_mismatch=True,allow_type_mismatch=True)
    ring_ref.movex(beam_length/4)
    o1_ring = ring_ref['o1']
    o2_ring = ring_ref['o2']
    c.add_port(
        name='optical_in',
        width=o1_ring.width,
        orientation=-90,
        center=(o1_ring.center[0] - (waveguide_spacing- x_span_resonator)/2, o1_ring.center[1] -50),
        layer=o1_ring.layer,
        port_type='optical'
    )
    c.add_port(
        name='optical_out',
        width=o2_ring.width,
        orientation=-90,
        center=(o2_ring.center[0] + (waveguide_spacing- x_span_resonator)/2, o2_ring.center[1] -50),
        layer=o2_ring.layer,
        port_type='optical'
    )
    copied_portin = c.ports.filter(regex='optical_in')[0].copy()
    copied_portout = c.ports.filter(regex='optical_out')[0].copy()
    copied_portin.name = 'optical_in_reverse'
    copied_portout.name = 'optical_out_reverse'
    copied_portin.orientation = 90
    copied_portout.orientation = 90
    r1 = gf.routing.route_single(
        c,
        port1=o1_ring,
        port2=copied_portin,
        cross_section=xs_routing,
        )
    r2 = gf.routing.route_single(
        c,
        port1=o2_ring,
        port2=copied_portout,
        cross_section=xs_routing,
    )
    bfs_ref = c << beam_fixed_support(size=(25,25), metal_offset=1)
    bfs_ref.connect("E1", beam_ref.ports["e1"],allow_width_mismatch=True,allow_layer_mismatch=True,allow_type_mismatch=True)

    make_ALD_PL_mask(c)
    c = tmp_merge_deep_etch_mask(c)
    c.flatten()
    
    return c 
c = beam_with_ring_resonator_3_combs(beam_length=70, beam_width=0.4,spring_length=15, spring_width=0.3,finger_num = 91, finger_num_sub=29,waveguide_spacing=50)
c.show()

In [ ]:
device_list = [
    beam_with_ring_resonator_3_combs(beam_length=100, beam_width=0.4,spring_length=15, spring_width=0.3,finger_num = 91, finger_num_sub=29),
    beam_with_ring_resonator_3_combs(beam_length=100, beam_width=0.5,spring_length=15, spring_width=0.3,finger_num = 91, finger_num_sub=29),
    beam_with_ring_resonator_3_combs(beam_length=150, beam_width=0.4,spring_length=15, spring_width=0.3,finger_num = 91, finger_num_sub=29),
    beam_with_ring_resonator_3_combs(beam_length=150, beam_width=0.5,spring_length=15, spring_width=0.3,finger_num = 91, finger_num_sub=29),
    beam_with_ring_resonator_3_combs(beam_length=70, beam_width=0.4,spring_length=15, spring_width=0.3,finger_num = 91, finger_num_sub=29),
    beam_with_ring_resonator_3_combs(beam_length=70, beam_width=0.5,spring_length=15, spring_width=0.3,finger_num = 91, finger_num_sub=29),
    beam_with_ring_resonator_3_combs(beam_length=70, beam_width=0.4,spring_length=15, spring_width=0.3,finger_num = 91, finger_num_sub=29),
]

In [ ]:
@gf.cell
def full_device_array_with_grating_couplers(device_list:list, spacing=1300):
    def device_array():
        c = gf.Component()
        device_refs = [c << d for d in device_list]
        for i, dev in enumerate(device_refs):
            dev.movex(i*spacing)
            # device_refs[i].movey(500*(-1)**i)
        c.ports = {}
        for i, dev in enumerate(device_refs):
            c.add_ports([dev.ports["optical_in"], dev.ports["optical_out"]])
        c.auto_rename_ports()
        return c
    c = gf.Component()
    my_coupler_spec = partial(my_coupler,coupler=gc_spec,waveguide_width=0.43)
    gc_array = gf.components.grating_coupler_array(my_coupler_spec, pitch=250, n=16, rotation=-90, with_loopback=True, cross_section=xs_routing).copy()
    create_deep_etch_mask(gc_array,'bbox',mask_offset=100,deep_etch_layer='PADDING')
    gc_array_ref = c << gc_array
    gc_array_ref.move((-1000,-1000))
    device_array_ref = c << device_array()


    # align device array ports to grating coupler array ports
    device_array_ref.movex(
        origin=device_array_ref.ports['o8'].center[0],
        destination=gc_array_ref.ports['o7'].center[0]
    )
    device_array_ref_ports = [
        gf.Port(port.name, center=port.center, width=port.width, orientation=port.orientation, layer=port.layer)
        for port in device_array_ref.ports
    ]

    f = gf.routing.route_bundle(
        c,
        gc_array_ref.ports,
        device_array_ref_ports,
        cross_section=xs_routing,
        sort_ports=True,
        start_straight_length=300,
        separation=15
    )
    routing_bboxes = [inst.bbox() for route in f for inst in route.instances]
    for bbox in routing_bboxes:
        c.add_polygon(gf.kdb.Polygon(bbox).sized(50e3,100e3,2), layer='PADDING')

    c = tmp_merge_deep_etch_mask(c)
    c.flatten()
    return c
c = full_device_array_with_grating_couplers(device_list)
c.show()

In [ ]:
c_printable = convert_to_printable(c,post_collection_layers=['MTOP','SHALLOW_ETCH','ALD_ETCH_EBL','ALD_ETCH_PL'])
c_printable.flatten()
c_printable.show()

In [ ]:
layout_final = gf.Component()
layout_final << die_with_alignment_marks(1.5e4,layers=['MARKER'])
for i in range(4):
    printable_layout = convert_to_printable(full_device_array_with_grating_couplers(device_list if i%2==0 else device_list),post_collection_layers=['MTOP','SHALLOW_ETCH','ALD_ETCH_EBL','ALD_ETCH_PL'])
    c_printable_ref = layout_final << printable_layout
    c_printable_ref.move(c_printable_ref.center,(-2000,-5250+3500*i))
etch_depth_arrays = [layout_final << etch_depth_array(layers=['DEEP_ETCH_PL','DEEP_ETCH_EBL','SHALLOW_ETCH'],frame_layer='MTOP') for _ in range(4)]
for i, eda in enumerate(etch_depth_arrays):
    eda.move(
        eda.center,
        (-8000 if i % 2 == 0 else 8000, -2500 if i // 2 == 0 else 2500),
    )

alignment_array = [
    layout_final << litho_caliper_array(
        types=["EBL","PL","EBL","EBL","PL","PL"], 
        layers=["DEEP_ETCH_EBL","DEEP_ETCH_PL","SHALLOW_ETCH","ALD_ETCH_EBL","ALD_ETCH_PL","MTOP"], 
        frame_layer='MTOP') for _ in range(4)]
for i, aa in enumerate(alignment_array):
    aa.move(
        aa.center,
        (-8000 if i % 2 == 0 else 8000, -3500 if i // 2 == 0 else 3500),
    )

layout_final.show()

In [ ]:
euler_test_spec = partial(euler_test, grating_coupler_spec=my_coupler_spec,cross_section=xs_routing)
gc_rib = partial(my_coupler,gc_spec,deep_layer="SHALLOW_ETCH")
def optical_test_array():
    c = gf.Component()
    (c << euler_bend_test_array(euler_test_spec)).move((0,0))
    (c << converter_test_array(partial(converter_test, grating_coupler_spec=gc_rib),pair_num_list=[10,20,50,60])).move((0,-500))
    (c << spiral_test(grating_coupler_spec=gc_rib,cross_section=xs_routing)).move((0,-1100))
    return c
(layout_final << optical_test_array()).move((3800,-4000))
(layout_final << optical_test_array()).move((3800,6200))
layout_final.show()

In [ ]:
def beam_test_array():
    c = gf.Component()
    for i in range(4):
        beam_ref = c << beam_test(width=5,length=50+i*40)
        beam_ref.move(((i%2)*1000,-(i//2)*600))
    return c
(layout_final << beam_test_array()).move((4200,-1100))
(layout_final << beam_test_array()).move((4200,2100))
layout_final.show()